In [0]:
# First I'm going to set up the environment a little more by importing some packages
from pyspark.sql.functions import avg, col, desc, regexp_replace, countDistinct, lower, udf, split, monotonically_increasing_id, row_number
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, Word2Vec
from pyspark.sql.types import DoubleType #DoubleType == Extra long float
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.window import Window

#from pyspark.ml import Pipeline

import numpy as np

In [0]:
#Importing Data from locally stored CSV downloaded from Kaggle at this link https://www.kaggle.com/datafiniti/womens-shoes-prices

# File location and type
file_location = "/FileStore/tables/Datafiniti_Womens_Shoes_Jun19.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

id,dateAdded,dateUpdated,asins,brand,categories,primaryCategories,colors,dimension,ean / upc,imageURLs,keys,manufacturer,manufacturerNumber,name,prices.amountMax,prices.amountMin,prices.availability,prices.color,prices.condition,prices.currency,prices.dateAdded,prices.dateSeen,prices.isSale,prices.merchant,prices.offer,prices.returnPolicy,prices.shipping,prices.size,prices.sourceURLs,sizes,sourceURLs
AWpyySsJAGTnQPR7wNt4,2019-05-01T09:27:22.000+0000,2019-05-01T09:27:22.000+0000,null,City Classified,"Clothing,Shoes,Womens Shoes,All Womens Shoes",Shoes,Black,null,656000000000,"https://i5.walmartimages.com/asr/f0dc6cbd-fa38-4322-8751-eb1bbd4320fb_1.6c0f69a4fdfc024886a2adb501d0499f.jpeg,https://i5.walmartimages.com/asr/20d97853-d308-4958-877b-03ba82ea4962_1.49ef2205cc3e74c0241c80afa06c59d7.jpeg,https://i5.walmartimages.com/asr/9222bac7-7c16-4e31-a7f9-dac3959ef17e_1.f8d1355184e1553a25d3da55b9fe180c.jpeg,https://i5.walmartimages.com/asr/1e235fd9-ae58-49c6-9faf-876123dff20e_1.c3aff616e45682bbae4f509761e358a9.jpeg,https://i5.walmartimages.com/asr/afacdff0-68c2-432b-b3f0-a83bc289768d_1.e14a27760016da81617cc1031a823255.jpeg","walmart.com-2013PXX8GMN7,656143772191,0656143772191",null,null,City Classified Stylish-S Women's Zipper Ankle Booties,29.95,29.95,true,Black,null,USD,null,2019-04-19T07:09:29Z,null,null,Online only,null,null,8,https://www.walmart.com/ip/City-Classified-Stylish-S-Women-s-Zipper-Ankle-Booties/734423266,8,https://www.walmart.com/ip/City-Classified-Stylish-S-Women-s-Zipper-Ankle-Booties/734423266
AWpyyyb3AGTnQPR7wN-u,2019-05-01T09:27:19.000+0000,2019-05-01T09:27:19.000+0000,null,Skechers,"Clothing,Shoes,Womens Shoes,All Womens Shoes",Shoes,Taupe,null,192000000000,null,"192283420528,0192283420528,walmart.com-21739WHUW8TK",null,null,"Skechers Women'S You Radiate, Walking",84.0,84.0,true,Taupe,null,USD,null,2019-04-24T09:14:11Z,null,null,Online only,null,null,6 M US,https://www.walmart.com/ip/Skechers-Women-S-You-Radiate-Walking/469752124,6 M US,https://www.walmart.com/ip/Skechers-Women-S-You-Radiate-Walking/469752124
AWpyzlajAGTnQPR7wOX8,2019-05-01T09:26:59.000+0000,2019-05-01T09:26:59.000+0000,null,Floral,"Clothing,Shoes,Womens Shoes,All Womens Shoes",Shoes,Black,null,709000000000,null,"walmart.com-1MXIR45EG0EO,708750137976,0708750137976",null,null,"FLORAL Pearl Women's Wide Width Evening Dress Shoes for Wedding, Prom, Dinner BLACK 12",49.95,49.95,true,Black,null,USD,null,2019-04-25T12:35:00.000Z,null,null,Online only,null,null,5,https://www.walmart.com/ip/FLORAL-Pearl-Women-s-Wide-Width-Evening-Dress-Shoes-for-Wedding-Prom-Dinner-BLACK-12/455903421,5,https://www.walmart.com/ip/FLORAL-Pearl-Women-s-Wide-Width-Evening-Dress-Shoes-for-Wedding-Prom-Dinner-BLACK-12/455903421
AWpyxomE0U_gzG0hkA1q,2019-05-01T09:26:56.000+0000,2019-05-01T09:26:56.000+0000,null,Jambu,"Clothing,Shoes,Womens Shoes,All Womens Shoes",Shoes,Black/Multi,null,884000000000,"https://i5.walmartimages.com/asr/bf04e7cf-0f7b-41d9-ba62-304121d9ef58_1.277114c5d75c38e5434303701fa372ff.jpeg,https://i5.walmartimages.com/asr/dba3658c-3349-4975-8e03-2d9544843c52_1.6fbb6bd702b333f5a0b0ed6b70868ec2.jpeg,https://i5.walmartimages.com/asr/ff269020-e097-451a-9813-b2fbc4dfe6c9_1.cc131be016b3baad5e0d14a9075a4f28.jpeg,https://i5.walmartimages.com/asr/f0a26477-7eae-4e2a-a855-0011b771daa1_1.46b938cd97bbd2710cb5de41388572bf.jpeg","883988336206,0883988336206,walmart.com-653BPY25H76W",null,null,Women's Jambu Amal Water Resistant Bootie,127.2,127.2,false,Black/Multi,null,USD,null,2019-04-30T19:49:00.000Z,null,null,Online only: 20% off,null,null,9.5 BM US,https://www.walmart.com/ip/Jambu-Women-s-AMAL-Water-Resistant-Ankle-Bootie/954498267,9.5 BM US,https://www.walmart.com/ip/Jambu-Women-s-AMAL-Water-Resistant-Ankle-Bootie/954498267
AWpyxChWJbEilcB6RhWx,2019-05-01T09:26:56.000+0000,2019-05-01T09:26:56.000+0000,null,Trotters,"Clothing,Shoes,Womens Shoes,All Womens Shoes",Shoes,White,null,887000000000,null,"887246520857,0887246520857,walmart.com-3HL720GQURLM",null,null,"trotters wom

### Doing some basic investigation of the data

In [0]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- dateAdded: timestamp (nullable = true)
 |-- dateUpdated: timestamp (nullable = true)
 |-- asins: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- primaryCategories: string (nullable = true)
 |-- colors: string (nullable = true)
 |-- dimension: string (nullable = true)
 |-- ean / upc: long (nullable = true)
 |-- imageURLs: string (nullable = true)
 |-- keys: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- manufacturerNumber: string (nullable = true)
 |-- name: string (nullable = true)
 |-- prices.amountMax: double (nullable = true)
 |-- prices.amountMin: double (nullable = true)
 |-- prices.availability: boolean (nullable = true)
 |-- prices.color: string (nullable = true)
 |-- prices.condition: string (nullable = true)
 |-- prices.currency: string (nullable = true)
 |-- prices.dateAdded: string (nullable = true)
 |-- prices.dateSeen: string (nullable = true)

In [0]:
display(df.groupBy('brand') \
        .agg(avg('`prices.amountMax`').alias("avgPricePerBrand")) \
        .sort(col("avgPricePerBrand")))

# Just by perusing the brands and average prices I've noticed TOTES with an average price of 5000 seems to be an obvious outlier here, I should also be brand frequency in my dataset. I don't want brands to be recognized as different because it came from a different retailer's website e.g. "Faded Glory" and "FADED GLORY" are the same brand but would be treated as different categories without preprocessing.

# The number of shoes listed per brand
display(df.groupBy('brand').count().sort(col('count')))

brand,avgPricePerBrand
TIME AND TRU,5.5
CTM,6.95
Alotta Knits,9.95
LAVRA,9.99
White Stag,10.87
FADED GLORY,11.38
Faded Glory,11.522307692307693
George,11.97
MYLEYON,11.98
Collections Etc,11.98


brand,count
Avanti,1
Mo Mo,1
Isa Tapia,1
MYLEYON,1
Aravon,1
Billabong,1
under-armour,1
Chiara Ferragni,1
Floopi,1
BUSSOLA,1


##Some processing notes

There are some inconsistencies in formatting of some of data like sizing which should be addressed by preprocessing before being fed into a model. This also requires some domain knowledge as the listings exist across different websites and from different manufacturers, for example "9.5 BM US" refers to the width "B" (where the later in the alphabet, the larger the shoe) and "M" meaning "Medium" as detailed here https://workplacesafetyshoes.com/what-does-bm-mean-in-shoes/. I make the simplyfing assumption that shoe width does not matter here in order to convert sizes into a numerical feature, removing all non-numeric characters from the "prices.size" feature. I also make the assumption that shoe sizes greater than 30 are in European sizes, and thus will convert them into US sizing.

I'm choosing to include Latent Semantic Analysis (LSA/SVD) as a part of my model, so I'm also going to do some cleaning of ingested text data to make a better semantic representation and fix some common formatting issues.

Because the dataset contains two price columns, I will average them into a single target column although they are usually the same.

note: because some of the columns contain a period i.e. "prices.color" or "prices.condition", in Spark I need to use a backtick to escape because the period symbol has special meaning as detailed here https://stackoverflow.com/questions/30359539/accessing-column-names-with-periods-spark-sql-1-3

In [0]:
#Averaging Price Columns, since these are usally the same and I want a single target variable
price_cols = [col('`prices.amountMax`'), col('`prices.amountMin`')]
average_func = sum(x for x in price_cols)/len(price_cols)
df = df.withColumn('pricesAvg', average_func)

#Num brands before processing
print(df.agg(countDistinct(df["brand"])).collect())

#replacing dashes with spaces and making all characters lower case so "FADED GLORY" and "Faded Glory", "ellie" and "Ellie", "Steve Madden" and "steve-madden", are equivalent with their counterparts
df = df.withColumn("brand", regexp_replace(df["brand"], "-", " "))
df = df.withColumn("brand", lower(df["brand"]))

#Num brands after processing
print(df.agg(countDistinct(df["brand"])).collect())


[Row(count(brand)=505)]
[Row(count(brand)=437)]


In [0]:
df = df.withColumn("sizeNumeric", regexp_replace(df['`prices.size`'],  "[^\\d.]", "")) # Removing all but numeric characters from the prices.size col
df = df.withColumn("sizeNumeric", df["sizeNumeric"].cast(DoubleType())) #converting String type to DoubleType (which is a long float)
df.select(df["sizeNumeric"]).show()

# Unfortunately processing doesn't have a way to account for european sizes, nor strings that contain multiple sizes i.e. "36 M EU / 6 BM US" becoming "366" as you can see when examining the distinct values of sizeNumeric.
# This inconsistency when recording sizes makes truly corrects size processing very difficult.

df.select(df["sizeNumeric"]).distinct().show(1000)

+-----------+
|sizeNumeric|
+-----------+
|        8.0|
|        6.0|
|        5.0|
|        9.5|
|       11.0|
|       11.0|
|        9.0|
|        6.0|
|        5.0|
|        8.0|
|        7.0|
|        6.0|
|        7.0|
|        5.5|
|        6.0|
|        8.0|
|        6.0|
|       12.0|
|        9.0|
|        6.0|
+-----------+
only showing top 20 rows

+-----------+
|sizeNumeric|
+-----------+
|        8.0|
|        7.0|
|       11.5|
|        3.5|
|        9.5|
|       9.51|
|      355.0|
|        4.5|
|     419.51|
|     1141.0|
|       42.0|
|     3555.5|
|        6.5|
|       44.0|
|       null|
|       62.0|
|       35.0|
|     365.56|
|       64.5|
|      409.0|
|       39.0|
|        1.0|
|      6.537|
|     398.59|
|       37.0|
|       36.0|
|       41.0|
|        7.5|
|        4.0|
|     376.57|
|       11.0|
|      103.0|
|      600.0|
|        8.5|
|       72.0|
| 1.112115E7|
|       14.0|
|       82.0|
|     387.58|
|   4110.511|
|        3.0|
|      366.0|
|       

In [0]:
#My processing strategy for sizing is to first convert the euro sizes on this https://www.zappos.com/c/shoe-size-conversion table to an appropriate US size. 
#Then, if the size is not between 3-15, it makes the sizes equal to the average women's size of 8.5 which seems to be the most common answer when googled 

@udf(returnType=DoubleType())
def euroToUSSize(input_num):
  #Converts Euro Sizes to US
  if input_num == 35.0:
    return 5
  elif input_num == 36.0:
    return 5.5
  elif input_num == 37.0:
    return 6.5
  elif input_num == 38.0:
    return 7.5
  elif input_num == 39.0:
    return 8.5
  elif input_num == 40.0:
    return 9.5
  elif input_num == 41.0:
    return 10.5
  elif input_num == 42.0:
    return 11.5
  elif input_num == 43.0: #extrapolating outside table
    return 12.5
  elif input_num == 44.0: #extrapolating outside table
    return 13.5
  elif input_num == 45.0: #extrapolating outside table
    return 14.5
  else:
    return input_num
  
@udf(returnType=DoubleType())
def otherSizingToAvg(input_num):
  #Removes sizes outside of standard US range
  if input_num is None:
    return 8.5
  elif (input_num < 15) and (input_num > 3):
    return input_num
  else:
    return 8.5
  
df = df.withColumn("sizeNumeric", euroToUSSize("sizeNumeric"))


In [0]:
# Because all rows are null or only have a single value or two, no reason to include these columns since they don't really add any additional information

df.select(df['`prices.condition`']).distinct().show()
df.select(df['`primaryCategories`']).distinct().show()
df.select(df['`prices.dateAdded`']).distinct().show()
df.select(df['`prices.Currency`']).distinct().show()
df = df.drop('`prices.condition`','`primaryCategories`','`prices.Currency`', '`prices.dateAdded`') 

+----------------+
|prices.condition|
+----------------+
|            null|
+----------------+

+--------------------+
|   primaryCategories|
+--------------------+
|               Shoes|
|Apparel & Accesso...|
+--------------------+

+----------------+
|prices.dateAdded|
+----------------+
|            null|
+----------------+

+---------------+
|prices.Currency|
+---------------+
|            USD|
+---------------+



In [0]:
df.select('manufacturer').distinct().show()

+------------------+
|      manufacturer|
+------------------+
|              null|
|        Comfy Feet|
|        Brinley Co|
|            Alfani|
|           Pleaser|
|             ASICS|
|        PleaserUSA|
|              VANS|
|          Bonnibel|
|Journee Collection|
|          Dyeables|
|             asics|
|           Alegria|
|       Pleaser USA|
|       Brinley Co.|
|       Easy Street|
|       ELLIE SHOES|
|           BEARPAW|
|       Dr. Martens|
|             Crocs|
+------------------+
only showing top 20 rows



In [0]:
# Cleaning color column
#"black/multi", "black / silver", "black and white" will all get tokenized differently
#df = df.withColumn("sizeNumeric", regexp_replace(df['`prices.size`'],  "[^\\d.]", ""))

@udf 
def cleanColor(input_color):
  input_color = input_color.lower()
  input_color = input_color.replace("/", " ")
  input_color = input_color.replace("-", " ")
  input_color = input_color.replace(" and ", " ")
  input_color = input_color.replace("polyurethane", "pu") #this is a shortened version so I want to make them recognized as the same
  input_color = " ".join(input_color.split()) #remove excess spaces i.e. "  " becomes " " 
  output_color = input_color.strip()
  return output_color

df = df.withColumn("cleanedColor", cleanColor("`prices.color`"))
df.select("cleanedColor").display()

cleanedColor
black
taupe
black
black multi
white
metallic multi soft leather
denim blue
black silver
black white combo
black silver


In [0]:
# Cleaing name column
# (this is basically non brand, non color/material info)

# The name of a shoe contains a lot of information, but it also contains information already included in other columns in the dataset like size, brand and color some examples here:
# "Steve Madden Women's Notice Rose Gold Ankle-High Leather Pump - 9M", "Not Rated Womens Slingbacks Sandals Size 8.5 New", "Women's Over Knee High Block Chunky Heel Thigh Heel Faux Suede Boots (FREE SHIPPING)"
# Before using the name data, I want to remove this unneccesary information

df = df.withColumn("nameNonNumeric", regexp_replace(df["name"],  "[\d-]", "")) #Removing all numeric characters to get rid of sizing

@udf
def cleanName(input_name, input_color, input_brand):
  input_low = input_name.lower()
  input_color = input_color.lower()
  for remove_str in ["women's", "womens", "women", "free shipping", "new", "size", "(", ")", '"', ".", ",", " bm ", " m ", " w ", " n ", " us ", " s ", " fa " " by "]: #removing some common terms that don't add additional info
    input_low = input_low.replace(remove_str, "")
  for remove_color in input_color.split(" "): #removing included color data in name column
    input_low = input_low.replace(remove_color.strip(), "")
  output_name = ' '.join([w for w in input_low.split() if len(w)>2]) #removing all substrings with less than 2 characters, along these lines https://stackoverflow.com/questions/24332025/remove-words-of-length-less-than-4-from-string
  output_name = output_name.replace(input_brand, "") #removing the brand name if its included as a part of the title (since we already have a brand column) e.g. "steve madden"
  return output_name.strip()

  
df = df.withColumn("cleanedName", cleanName("nameNonNumeric", "cleanedColor", "brand"))
df.select("cleanedName").display()

cleanedName
stylishs zipper ankle booties
you radiate walking
pearl wide width evening dress shoes for wedding prom dinner
amal water resistant bootie
blast iii bootblack quillus
walk hart quarter strap wedge sandal
lulu art toe thong slide sandals
thhspectrum prism heel and with plain vamp
thhspectrum prism heel and with plain vamp
thhspectrum prism heel and with plain vamp


In [0]:
# Turning brand into a numerical column (machine learning models read numerical data)
# https://datascience.stackexchange.com/questions/6268/how-to-convert-categorical-data-to-numerical-data-in-pyspark
brand_indexer = StringIndexer(inputCol="brand", outputCol="brandIndex", handleInvalid="keep")
manufact_indexer = StringIndexer(inputCol="manufacturer", outputCol="manufacturerIndex", handleInvalid="keep")
df = brand_indexer.fit(df).transform(df)
df = manufact_indexer.fit(df).transform(df)

In [0]:
# One hot encoding brand data, this turns  into distinct features
ohe = OneHotEncoder(inputCol="brandIndex", outputCol="brandOHE")
ohe_model = ohe.fit(df)
df = ohe_model.transform(df)

In [0]:
#Splitting short descriptive sentences on individual words (neccesary to use word2vec)
#https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.split.html

split_color = df.select(split(df.cleanedColor, '[ ]', -1).alias('splitColor'))
split_name = df.select(split(df.cleanedName, '[ ]', -1).alias('splitName'))

In [0]:
#Creating word embeddings
colors2vec = Word2Vec(inputCol='splitColor', vectorSize=50, outputCol='colorVec')
color_vec_model = colors2vec.fit(split_color)
color_out = color_vec_model.transform(split_color)
name2vec = Word2Vec(inputCol='splitName', vectorSize=50, outputCol='nameVec')
name_vec_model = name2vec.fit(split_name)
name_out = name_vec_model.transform(split_name)
#name_out = name_out.withColumnRenamed("Word2Vec_68eff28684ff__output","nameVec") 

In [0]:


# https://stackoverflow.com/questions/42853778/add-a-column-from-another-dataframe
# since there is no common column between these two dataframes add row_index so that it can be joined
color_out=color_out.withColumn('row_index', row_number().over(Window.orderBy(monotonically_increasing_id())))
name_out=name_out.withColumn('row_index', row_number().over(Window.orderBy(monotonically_increasing_id())))
df=df.withColumn('row_index', row_number().over(Window.orderBy(monotonically_increasing_id())))

df = df.join(color_out, on=["row_index"])
df = df.join(name_out, on=["row_index"]).drop("row_index")

In [0]:
# Creating a feature vector (this is how spark wants data processed for modeling)
# https://stackoverflow.com/questions/33428589/pyspark-and-pca-how-can-i-extract-the-eigenvectors-of-this-pca-how-can-i-calcu/33500704#33500704
assembler = VectorAssembler(inputCols=['colorVec', 'nameVec', 'sizeNumeric', 'brandOHE'], outputCol="features").setParams(handleInvalid="skip")
df_assembler = assembler.transform(df)

In [0]:
# Splitting the data into test and training partitions
(trainingData, testData) = df_assembler.randomSplit([0.9, 0.1])

In [0]:
# Defining a GLR model
glr = GeneralizedLinearRegression(labelCol="pricesAvg", featuresCol="features", family="gaussian", link="identity", maxIter=10, regParam=0.05)

# TO DO: 
# Chain indexer and GBT in a Pipeline
# pipeline = Pipeline(stages=[featureIndexer, glr])

# Train model.  This also runs the indexer.
model = glr.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "pricesAvg", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="pricesAvg", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

+------------------+---------+--------------------+
|        prediction|pricesAvg|            features|
+------------------+---------+--------------------+
|24.052127446794387|    19.95|(538,[0,1,2,3,4,5...|
|21.652254294931396|    19.95|(538,[0,1,2,3,4,5...|
|21.652242255148956|    19.95|(538,[0,1,2,3,4,5...|
|38.963401434052564|    45.95|(538,[0,1,2,3,4,5...|
|16.617218807754597|    32.95|(538,[0,1,2,3,4,5...|
+------------------+---------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 77.3689


In [0]:
predictions.select("prediction", "pricesAvg", "features").display()

prediction,pricesAvg,features
27.917573753482472,19.95,"Map(vectorType -> sparse, length -> 538, indices -> List(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101), values -> List(0.0018733832985162735, -0.008545886725187302, 1.721688313409686E-4, 0.00958319753408432, 0.01081122551113367, 0.004682426806539297, 0.014800895005464554, -0.0065243858844041824, 0.006391321774572134, 0.0033613466657698154, 0.005955354310572147, 0.014753934927284718, -3.23296757414937E-4, -0.009912148118019104, -0.015050477348268032, -0.00331605551764369, 0.001080218469724059, 0.005644699092954397, -0.01081375777721405, -0.009122033603489399, 0.0030955977272242308, 0.010620842687785625, -0.001975075574591756, -0.0031202505342662334, 0.008804328739643097, -0.002063452498987317, 0.01780075393617153, -0.0037344186566770077, -0.001976387808099389, -0.00616361852735281, 0.009274812415242195, -0.0019155030604451895, -0.019925478845834732, 0.004655311815440655, -0.007682591676712036, 0.0032270713709294796, 0.01057992223650217, 0.005231646355241537, -0.00213070260360837, -0.004782525356858969, -0.004735742695629597, 0.006391800940036774, 0.007315432652831078, 0.003385361284017563, -0.015969661995768547, -0.0032499327789992094, -0.0016937576001510024, -0.007898127660155296, 0.0035276925191283226, -0.008063029497861862, -0.3087817490100861, 0.12170251607894898, 0.34057210683822636, -0.35830606035888196, -0.2680667132139206, -0.10521509423851967, 0.18417724892497064, -0.045926146954298024, 0.09119999036192894, 0.18388967365026476, 0.14316673651337625, -0.0677706990391016, -0.17010345458984377, -0.1883759751915932, -0.07760521024465561, -0.17257910147309305, -0.08901824280619622, 0.22523266077041626, -0.10835646688938141, 0.15879732370376587, 0.4146757379174233, -0.37608416676521306, 0.1781229231506586, 0.00765950083732605, 0.23690568506717682, 0.24675493661779913, 0.04647306837141514, -0.1369980290532112, 0.07452093213796616, 0.24884115010499955, -0.13159706927835943, -0.03368637403473258, 0.04231745451688767, -0.0514879310503602, -0.4312586307525635, 0.03141115307807923, 0.15435900241136552, -0.04351872224360705, 0.07574303764849902, 0.11214164197444916, -0.16610010601580144, -0.21436323106754573, -0.1553720870986581, 0.03177352100610733, -0.32765015810728076, 0.061409581219777465, 0.549892944097519, 0.1479369502514601, 0.09070689380168916, 0.08405585289001466, 6.5, 1.0))"
28.904839171670723,19.95,"Map(vectorType -> sparse, length -> 538, indices -> List(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101), values -> List(-0.10313594341278076, -0.08303477615118027, -0.018077973276376724, -0.022419875487685204, 0.015514320693910122, 0.037294451147317886, 0.127144455909729, -0.017608556896448135, -0.013790504075586796, 0.020776724442839622, 0.09174709767103195, 0.1737145185470581, 0.07523898780345917, -0.131269171833992, -0.07157285511493683, -0.03833029419183731, -0.061308663338422775, 0.0040510427206754684, -0.0738457515835762, 0.08920405060052872, -0.04132978245615959, -0.11354542523622513, 0.031213894486427307, -0.08195079863071442, -0.02381121926009655, 0.16387180984020233, 0.008059078827500343, -0.0769212618470192, -0.1487133651971817, -0.02443038672208786, 0.024783669039607048, -0.0572422593832016, -0.056352075189352036, 0.03464560583233833, -0.01648070104420185, 0.08595573

help from https://towardsdatascience.com/your-first-apache-spark-ml-model-d2bb82b599dd
and https://mingchen0919.github.io/learning-apache-spark/